## Importing the necessary modules

In [1]:
from json import loads
from random import choice, shuffle
from string import punctuation
from keras import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, Softmax
from keras.optimizers import Adam
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from numpy import array
'''
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')
'''


'\nnltk.download("punkt")\nnltk.download("wordnet")\nnltk.download(\'omw-1.4\')\n'

## Loading the dataset

In [2]:
data = loads(open("dataset/intents.json", encoding="utf-8").read())


## Using NLP to clean the data

In [3]:
lemmatizer = WordNetLemmatizer()

# Each list to create
words = [] # tokenized words of sentences in patterns
classes = [] # tags
doc_X = [] # patterns
doc_y = [] # tags ocurring number of times wrt patterns

'''Looping through all the intents and tokenizing each patterns and
appending tokens to words, patterns and associated tag to
their associated list'''
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    # add the tag to the classes if it's not there already
    if intent["tag"] not in classes:
        classes.append(intent["tag"])


In [4]:
# lemmatizing all the words in the vocab and
# converting them to lowercase if the words
# don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in punctuation]


In [5]:
'''sorting the vocab and classes in alphabetical order
and taking the set to ensure no duplicates occur'''
words = sorted(set(words))
classes = sorted(set(classes))


In [6]:
print("lenght of words: ", len(words))
print("length of doc_X", len(doc_X))
print("length of doc_y", len(doc_y))
print("length of classes", len(classes))


lenght of words:  115
length of doc_X 111
length of doc_y 111
length of classes 23


## Creating training and test sets

In [7]:
# list for training data
training = []

# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    # lemmatizing the sentence and converting to lowercase
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        # one hot encoding the words if they appear in the text
        if word in text:
            bow.append(1)
        else:
            bow.append(0)
    output_row = [0] * len(classes)
    # marking the index of class that the current pattern is associated to as 1
    output_row[classes.index(doc_y[idx])] = 1
    # adding the one hot encoded BoW and associated classes to training
    training.append([bow, output_row])


In [8]:
# shuffling the data and convert it to a numpy array
shuffle(training)
training = array(training, dtype=object)


In [9]:
# splitting the features and target labels
train_X = array(list(training[:, 0])) # features
train_y = array(list(training[:, 1])) # target labels


In [10]:
print(train_X)


[[0 0 1 ... 0 0 0]
 [0 0 1 ... 0 1 0]
 [0 0 1 ... 0 1 0]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [11]:
print(train_y)


[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]]


In [12]:
print(f"Shape of train_X: {train_X.shape}")
print(f"Shape of train_y: {train_y.shape}")


Shape of train_X: (111, 115)
Shape of train_y: (111, 23)


# Building deep learning model

In [13]:
# defining some parameters
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])
print("input_shape: ", input_shape)
print("output_shape: ", output_shape)


input_shape:  (115,)
output_shape:  23


In [14]:
# defining the model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation=LeakyReLU(alpha=0.3)))
model.add(Dropout(0.25))
model.add(Dense(64, activation=LeakyReLU(alpha=0.3)))
model.add(Dropout(0.25))
model.add(Dense(output_shape, activation=Softmax()))


In [15]:
# compiling the model
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=["accuracy"])


In [16]:
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               14848     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 23)                1495      
                                                                 
Total params: 24,599
Trainable params: 24,599
Non-trainable params: 0
_________________________________________________________________
None


## Training the model

In [17]:
model.fit(x=train_X, y=train_y, epochs=200)


Epoch 1/200


2022-11-28 15:52:53.593887: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


4/4 [==============================] - 0s 1ms/step - loss: 3.0970 - accuracy: 0.0901
Epoch 2/200
4/4 [==============================] - 0s 1ms/step - loss: 2.5940 - accuracy: 0.4234
Epoch 3/200
4/4 [==============================] - 0s 1ms/step - loss: 2.0939 - accuracy: 0.6126
Epoch 4/200
4/4 [==============================] - 0s 1ms/step - loss: 1.4753 - accuracy: 0.7027
Epoch 5/200
4/4 [==============================] - 0s 2ms/step - loss: 0.9489 - accuracy: 0.8378
Epoch 6/200
4/4 [==============================] - 0s 1ms/step - loss: 0.5648 - accuracy: 0.9009
Epoch 7/200
4/4 [==============================] - 0s 1ms/step - loss: 0.3839 - accuracy: 0.9099
Epoch 8/200
4/4 [==============================] - 0s 2ms/step - loss: 0.2823 - accuracy: 0.9279
Epoch 9/200
4/4 [==============================] - 0s 2ms/step - loss: 0.1567 - accuracy: 0.9640
Epoch 10/200
4/4 [==============================] - 0s 2ms/step - loss: 0.1220 - accuracy: 0.9910
Epoch 11/200
4/4 [=======================

# Functions for getting results

In [18]:
def clean_text(text):
    """function to tokenize and then lemmatize and
    returning the tokens array"""
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens


def bag_of_words(text, vocab):
    """function to get the bag of words and one hot
    encoding the words if they appear in the text and
    returning numpy array of bow"""
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for tok in tokens:
        for idx, word in enumerate(vocab):
            if word == tok:
                bow[idx] = 1
    return array(bow)


def pred_class(text, vocab, labels):
    """function to predict the class"""
    bow = bag_of_words(text, vocab)
    result = model.predict(array([bow]))[0]
    thresh = 0.2
    y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for res in y_pred:
        return_list.append(labels[res[0]])
    return return_list


def get_response(intents_list, intents_json):
    """taking the predicted class and returning
    a random response from the intents.json file"""
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = choice(i["responses"])
            break
    return result


## Testing the model

In [ ]:
from datetime import datetime
import platform
import pyttsx3
import speech_recognition as sr

now = datetime.now()
engine = pyttsx3.init()
r = sr.Recognizer()
mic = sr.Microphone()

if platform.system() == "Windows":  # for Windows
    engine.setProperty(
        'voice', r"HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0")
elif platform.system() == "Darwin":  # for macOS
    engine.setProperty(
        'voice', "com.apple.speech.synthesis.voice.samantha")
else:
    pass

TEXT = "Anuradha is ready to chat! (say 'exit' to quit)"
print(TEXT)
engine.say(TEXT)
engine.runAndWait()

while True:
    with mic as source:
        r.adjust_for_ambient_noise(source)
        r.dynamic_energy_threshold = True
        print("Say Now")
        audio = r.record(source, duration=3)
    try:
        message = r.recognize_google(audio, language="en-IN") # message = input("You : ")
        intents = pred_class(message, words, classes)
        result = get_response(intents, data)
        if message == 'exit':
            TEXT = "Bye! take care"
            break
        elif result == "date":
            TEXT = now.strftime(r'%d/%m/%Y')
        elif result == "time":
            TEXT = now.strftime(r'%H:%M:%S')
        else:
            TEXT = result
    except sr.UnknownValueError:
        TEXT = "Sorry, I didn't get that"
    except sr.RequestError as e:
        TEXT = "Sorry, can't connect to the service"
    finally:
        try:
            print("You :", message)
        except NameError:
            pass
        finally:
            print("Anuradha :", TEXT)
            engine.say(TEXT)
            engine.runAndWait()
